In [139]:
import pandas as pd
from nltk.tokenize import sent_tokenize

In [140]:
original_text = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\unigram_data.h5", key="original")

In [141]:
ordered_text = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5", key="text")

In [142]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize.nist import NISTTokenizer

def replace_quotes(x):
    for match in re.findall(" “.+?[”'\1{2}]", x):
        x = x.replace(match, "")

    return x

def preprocess(x):
    tk = NISTTokenizer()
    x = replace_quotes(x)
    x = sent_tokenize(x)
    x = [[word for word in tk.tokenize(sentence) if word not in string.punctuation] for sentence in x]
    
    return x

platforms = ["Pitchfork", "Guardian", "Spectrum", "NME"]

for platform in platforms:
    
    original_text.loc[:, platform] = original_text.loc[:, platform].apply(preprocess)

In [143]:
original_text = original_text.melt(id_vars=["Artist", "Album"], value_vars=["Pitchfork", "Guardian", "Spectrum", "NME"], var_name="Platform", value_name="Text")

In [144]:
original_text["Full Text"] = original_text.loc[:, "Text"].apply(lambda x: "|".join(["|".join(sent) for sent in x]))

In [145]:
original_text = original_text.drop_duplicates(subset=["Full Text"]).drop(columns=["Full Text"])

In [146]:
artist_album_sort = ordered_text.loc[:, ["Artist", "Album"]]
original_text.loc[:, "Artist"] = pd.Categorical(original_text.loc[:, "Artist"], artist_album_sort.loc[:, "Artist"].unique())
original_text.loc[:, "Album"] = pd.Categorical(original_text.loc[:, "Album"], artist_album_sort.loc[:, "Album"].unique())
original_text.loc[:, "Platform"] = pd.Categorical(original_text.loc[:, "Platform"], ["Pitchfork", "Guardian", "Spectrum", "NME"])
original_text = original_text.sort_values(["Platform", "Album", "Artist"]).reset_index(drop=True)

In [147]:
original_text["Review id"] = [x for x in range(original_text.shape[0])]

In [148]:
original_text.loc[:, "Text"] = original_text.apply(lambda x: list(x))

In [149]:
original_text = original_text.astype({"Artist": str,
                    "Album": str,
                    "Platform": str,
                    "Text": str})

In [150]:
original_text.to_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5", key="sentence", format="table")

In [151]:
original_text

,Artist,Album,Platform,Text,Review id
0,sleaford mods,key markets,Pitchfork,"[['The', 'title', 'of', 'Key', 'Markets—', 'so...",0
1,black midi,hellfire,Pitchfork,"[['The', 'preposterously', 'talented', 'Englis...",1
2,kanye west,yeezus,Pitchfork,"[['Marking', 'a', 'blunt', 'break', 'with', 't...",2
3,kehlani,it was good until it wasn't,Pitchfork,"[['The', 'cloudy', 'grooves', 'of', 'the', 'Oa...",3
4,blood orange,negro swan,Pitchfork,"[['Dev', 'Hynes’', 'fourth', 'album', 'as', 'B...",4
...,...,...,...,...,...
709,father john misty,fear fun,NME,"[['Having', 'officially', 'taken', 'leave', 'o...",709
710,beyonce,4,NME,"[['It’s', 'not', 'been', 'a', 'vintage', 'year...",710
711,toro y moi,anything in return,NME,"[['Chazwick', 'Bundick', 'aka', 'Toro', 'Y', '...",711
712,james blake,assume form,NME,"[['James', 'Blake', 'James', 'Blake', 'has', '...",712
